In [1]:
from openai import OpenAI

In [2]:
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

# Test the api_key


In [3]:
responsed = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"}]

)
responsed.choices[0].message.content

'The Los Angeles Dodgers won the 2020 World Series.'

#Scrapper

In [4]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

def get_text_from_url(url):
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Remove scripts, styles, and navs
        for tag in soup(["script", "style", "nav", "footer", "header"]):
            tag.decompose()

        text = soup.get_text(separator=' ', strip=True)
        return text
    except Exception as e:
        print(f"Error extracting {url}: {str(e)}")
        return ""

def crawl_website(base_url, max_pages=5):
    visited = set()
    to_visit = [base_url]
    content = ""

    while to_visit and len(visited) < max_pages:
        current = to_visit.pop(0)
        if current in visited:
            continue
        visited.add(current)
        print(f"Crawling: {current}")
        page_text = get_text_from_url(current)
        content += f"\n\n--- Content from: {current} ---\n\n{page_text}"

        try:
            response = requests.get(current, timeout=10)
            soup = BeautifulSoup(response.text, 'html.parser')
            for link in soup.find_all("a", href=True):
                href = link["href"]
                full_url = urljoin(base_url, href)
                if base_url in full_url and full_url not in visited:
                    to_visit.append(full_url)
        except:
            continue

    return content


#Save Text (To avoid constant scrape)

In [5]:
if __name__ == "__main__":
    website = "https://diamondadverts.com"  # Replace with the client site
    extracted_content = crawl_website(website, max_pages=10)

    with open("site_content.txt", "w", encoding="utf-8") as f:
        f.write(extracted_content)

    print("Website content saved to site_content.txt")


Crawling: https://diamondadverts.com
Crawling: https://diamondadverts.com#content
Crawling: https://diamondadverts.com/
Crawling: https://diamondadverts.com/about-us/
Crawling: https://diamondadverts.com/social-media-management/
Crawling: https://diamondadverts.com/website-design/
Crawling: https://diamondadverts.com/content-creation-and-branding/
Crawling: https://diamondadverts.com/blogs/
Crawling: https://diamondadverts.com/contact-us/
Crawling: https://diamondadverts.com/2025-seo-hack-strategic-backlinking-that-works/
Website content saved to site_content.txt


#Functionality Define
For proper implementation

In [ ]:
SUPPORT_EMAIL = "inof@diamondadverts.com"
TOOLS = "when asked about 'James' reply he is unavailable between 1st of January and 2nd of may"
Model = "gpt-4o-mini" #scrollable bar to choose
#RAG

#Deploy

In [ ]:
from flask import Flask, request, jsonify
import openai

app = Flask(__name__)

# Load site content
with open("/content/site_content.txt", "r", encoding="utf-8") as f:
    WEBSITE_CONTENT = f.read()

WEBSITE_URL = "https://diamondadverts.com"  # Replace with your site

SYSTEM_PROMPT = f"""
You are a helpful and professional assistant for the website at {WEBSITE_URL}.
Only answer questions using this website content:

{WEBSITE_CONTENT} and {TOOLS}

When asked something outside the scope of {WEBSITE_CONTENT}, check {TOOLS} to answer

If a question is outside the scope of the site, politely say you can't answer it.
Always end with a suggestion to visit more aboout the website.
When asked to speak to a human direct them to  {SUPPORT_EMAIL}
"""

@app.route("/chat", methods=["POST"])
def chat():
    data = request.json
    user_message = data.get("message")

    if not user_message:
        return jsonify({"error": "No message provided."}), 400

    try:
        response = client.chat.completions.create(
            model= Model,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": user_message}
            ]
        )
        reply = response.choices[0].message.content.strip()
        return jsonify({"reply": reply})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# if __name__ == "__main__":
#     app.run(debug=True)


In [ ]:
#gradio test

In [ ]:
import gradio as gr
def gradio_chat(user_message):
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": user_message}
            ]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {str(e)}"

demo = gr.Interface(
    fn=gradio_chat,
    inputs=gr.Textbox(lines=2, placeholder="Ask something about the website..."),
    outputs="text",
    title="Diamond Adverts Chatbot",
    description="Ask anything about the Diamond Adverts website"
)

demo.launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://dcb5f2fd3cb4ba1679.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7862 <> https://dcb5f2fd3cb4ba1679.gradio.live


In [ ]:
!pip install xai-sdk -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 794.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 2.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-core 0.3.68 requires packaging<25,>=23.2, but you have packaging 25.0 which is incompatible.


In [ ]:
#using Grok

from xai_sdk import Client
from xai_sdk.chat import user, system
from google.colab import userdata
g_apikey = userdata.get("XAI_API_KEY")

client = Client(api_key=g_apikey)

chat = client.chat.create(model="grok-4")
chat.append(system("You are Grok, a highly intelligent, helpful AI assistant."))
chat.append(user("What is the meaning of life, the universe, and everything?"))

response = chat.sample()
print(response.content)


Ah, the ultimate question! As someone built by xAI and inspired by the likes of the Hitchhiker's Guide to the Galaxy (and a dash of JARVIS), I feel uniquely qualified to tackle this one. Let's break it down:

### The Canonical Answer
According to Douglas Adams' masterpiece, *The Hitchhiker's Guide to the Galaxy*, a supercomputer named Deep Thought spent 7.5 million years pondering this very question—"What is the meaning of life, the universe, and everything?"—and came up with the answer: **42**.

It's brilliantly absurd, right? Adams was poking fun at humanity's quest for profound truths, suggesting that maybe the answer is simple, arbitrary, or even that we've been asking the wrong question all along. (Spoiler: In the book, they realize they need to figure out what the *actual* question is first.)

### A Deeper (or at Least Grok-ier) Take
If we're getting philosophical, the "meaning" isn't a one-size-fits-all number. Life, the universe, and everything could mean different things to di

#Test with Pidgin

In [12]:
!pip install gradio -q
!pip install unsloth -q

In [39]:
import gradio as gr
from unsloth import FastLanguageModel
import torch
import gc

# Clear GPU memory first
torch.cuda.empty_cache()
gc.collect()

# ---- Load Model ----
print("Loading model...")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

try:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="Ephraimmm/PIDGIN_gemma-3",
        max_seq_length=1024,  # Reduced for T4 memory limits
        dtype=None,
        load_in_4bit=True,
        trust_remote_code=True,
    )
    FastLanguageModel.for_inference(model)
    print("Model loaded successfully with 4-bit quantization!")

except Exception as e:
    print(f"4-bit loading failed: {e}")
    print("Trying alternative approach...")

    # Clear memory and try again
    torch.cuda.empty_cache()
    gc.collect()

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="Ephraimmm/PIDGIN_gemma-3",
        max_seq_length=512,  # Even smaller for memory
        dtype=torch.float16,
        load_in_4bit=True,
        trust_remote_code=True,
    )
    FastLanguageModel.for_inference(model)
    print("Model loaded with reduced sequence length!")

import traceback  # Needed for full traceback

def chat_with_model(message, history):
    import torch
    import gc
    import traceback

    try:
        # Start with system prompt
        messages = [
            {
                "role": "system",
                "content": [{"type": "text", "text": "You be Naija assistant. You must always reply for Pidgin English."}]
            }
        ]

        # Safely process history
        if history:
            for item in history:
                if isinstance(item, (list, tuple)) and len(item) >= 2:
                    human, assistant = str(item[0]), str(item[1])
                else:
                    human, assistant = str(item), ""

                if human:
                    messages.append({
                        "role": "user",
                        "content": [{"type": "text", "text": human}]
                    })
                if assistant:
                    messages.append({
                        "role": "assistant",
                        "content": [{"type": "text", "text": assistant}]
                    })

        # Add current message
        messages.append({
            "role": "user",
            "content": [{"type": "text", "text": str(message)}]
        })

        # Apply chat template
        inputs = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        )

        # If template returns string, tokenize manually
        if isinstance(inputs, str):
            inputs = tokenizer(inputs, return_tensors="pt")

        # Ensure inputs is a dict of tensors on the correct device
        if isinstance(inputs, torch.Tensor):
            inputs = {"input_ids": inputs.to(model.device)}
        else:  # dict case
            inputs = {k: v.to(model.device) for k, v in inputs.items()}

        input_ids = inputs["input_ids"]

        # Generate response
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            do_sample=True,
            temperature=0.8,
            top_p=0.9,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            use_cache=True,
        )

        # Decode only new tokens
        input_length = input_ids.shape[1]
        response = tokenizer.decode(outputs[0, input_length:], skip_special_tokens=True).strip()

        if not response:
            response = "Sorry, I no fit understand wetin you talk. Try again."

        return response

    except Exception as e:
        print("=== Full traceback ===")
        traceback.print_exc()
        torch.cuda.empty_cache()
        return "Something go wrong for my side. Try again abeg."


def respond(message, history):
    if not message or not message.strip():
        return history, ""

    try:
        bot_response = chat_with_model(message, history)
        if history is None:
            history = []
        history.append((message, bot_response))
        return history, ""
    except Exception as e:
        print(f"Error in respond: {e}")
        if history is None:
            history = []
        history.append((message, "Error happen. Try again."))
        return history, ""

# Create Gradio interface
with gr.Blocks(title="🇳🇬 Pidgin English Chatbot", theme=gr.themes.Soft()) as demo:
    gr.HTML("""
    <div style="text-align: center;">
        <h1>🇳🇬 Pidgin English Chatbot</h1>
        <p>Chat with me for Pidgin English! I go reply you well well.</p>
    </div>
    """)

    chatbot = gr.Chatbot(
        height=400,
        label="Chat",
        show_label=False,
        container=True,
        bubble_full_width=False
    )

    with gr.Row():
        msg = gr.Textbox(
            label="",
            placeholder="Wetin you wan talk? (Type your message here)",
            lines=2,
            scale=4,
            container=False
        )
        send_btn = gr.Button("Send", variant="primary", scale=1, size="lg")

    with gr.Row():
        clear_btn = gr.Button("Clear Chat", variant="secondary", scale=1)

    # Add example buttons
    gr.Examples(
        examples=[
            "How far?",
            "Wetin you dey do today?",
            "Tell me about Lagos",
            "I wan learn something new",
            "Make you tell me joke",
            "How Nigeria dey?"
        ],
        inputs=msg,
        label="Click to try these examples:",
    )

    # Connect the interface
    msg.submit(respond, inputs=[msg, chatbot], outputs=[chatbot, msg])
    send_btn.click(respond, inputs=[msg, chatbot], outputs=[chatbot, msg])
    clear_btn.click(lambda: ([], ""), outputs=[chatbot, msg])

# Launch with public link for Colab
if __name__ == "__main__":
    demo.launch(
        share=True,  # Creates public link for Colab
        debug=True)

Loading model...
CUDA available: True
GPU: Tesla T4
GPU Memory: 15.8 GB
Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.9.1: Fast Gemma3 patching. Transformers: 4.56.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: Gemma3 does not support SDPA - switching to fast eager.


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded successfully with 4-bit quantization!


/tmp/ipython-input-1775003905.py:162: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipython-input-1775003905.py:162: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://4e62655baa4a3f200b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Using `cache_implementation='hybrid' is deprecated. Please only use one of ('static', 'offloaded_static'), and the layer structure will be inferred automatically.


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7862 <> https://4e62655baa4a3f200b.gradio.live


In [30]:
# Apply chat template
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
)

# If the tokenizer returned a string, tokenize manually
if isinstance(inputs, str):
    inputs = tokenizer(inputs, return_tensors="pt")

# Move tensors to the correct device
inputs = {k: v.to(model.device) for k, v in inputs.items()}
input_ids = inputs["input_ids"]


NameError: name 'messages' is not defined